# TD learning; the agent that can see

# Remember to check the number of samples for alpha and beta

now I'm gonna add numbers to the model. 

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import scipy.stats as stats
import seaborn as sns
from sklearn.metrics import confusion_matrix
# np.random.seed(42)
from joblib import Parallel, delayed
import matplotlib.tri as tri
import matplotlib.colors as mcolors
from scipy.interpolate import griddata
from scipy.interpolate import RBFInterpolator
import matplotlib.ticker as mticker
import itertools
from sklearn.metrics import r2_score
import optuna

# important directories

In [2]:
folder_path = 'data_risk_added_epileptic'


output_dir_model_evaluation = "13_RL_agent_TDlearn_output"
os.makedirs(output_dir_model_evaluation, exist_ok=True)

output_dir_plots = os.path.join(output_dir_model_evaluation, "plots")
os.makedirs(output_dir_plots, exist_ok=True)

output_dir_model_behavior = os.path.join(output_dir_model_evaluation, "model_behavior")
os.makedirs(output_dir_model_behavior, exist_ok=True)

In [3]:

dataframes = [pd.read_csv(os.path.join(folder_path, file)) for file in os.listdir(folder_path) if file.endswith('.csv')]


n_participant = len(dataframes)
print(f"there are {n_participant} participants.")


dataframes[2]

there are 7 participants.


,arrowRT,distribution,interTrialInterval,outcome,myCard,yourCard,spaceRT,totalReward,trialIndex,trialType,choice,block,timeoutRepeat,is_within_IQR,risk
0,2135,uniform,835,win,4,7,14079,10.5,0,response,arrowdown,1,0,0,0.375
1,1203,uniform,926,win,9,4,1804,11,1,response,arrowup,1,0,1,0.000
2,1035,uniform,934,win,1,9,766,11.5,2,response,arrowdown,1,0,1,0.000
3,827,uniform,991,lose,2,1,1189,11,3,response,arrowdown,1,0,1,0.125
4,1306,uniform,970,win,9,3,1887,11.5,4,response,arrowup,1,0,1,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
268,939,uniform,773,win,4,6,802,87,24,response,arrowdown,4,0,1,0.375
269,1210,high,811,win,6,8,1345,87.5,122,response,arrowdown,4,0,1,0.385
270,275,high,838,win,3,5,254,88,123,response,arrowdown,4,0,1,0.071
271,907,uniform,992,win,2,4,774,88.5,14,response,arrowdown,4,0,1,0.125


### I want to make participant file name for the model_evaluation.csv and that is I'm gonna take each data name task_data_07_11_2024_17_23_43.xlsx and extract "07_11_2024_17_23_43" and this should be the participant name in the dataset. 

In [4]:
participants = [os.path.splitext(file)[0].replace("task_data_", "")
    for file in os.listdir(folder_path) if file.endswith('.csv')]

# policy initilization for the model
now I need to find the prior policy amounts. for that I am going to put the percentage of downarrow and up arrow for each distribution.

In [5]:
df_combined = pd.concat(dataframes, ignore_index=True)

df_combined = df_combined[df_combined['outcome'].str.lower() != 'na'].reset_index(drop=True)
 

desired_order = ["uniform", "low", "high"]  


cards_sorted = sorted(df_combined["myCard"].unique())
dist_sorted = [d for d in desired_order if d in df_combined["distribution"].unique()]
choice_sorted = sorted(df_combined["choice"].unique())


card_idx = {card: i for i, card in enumerate(cards_sorted)}
dist_idx = {dist: i for i, dist in enumerate(dist_sorted)}
choice_idx = {choice: i for i, choice in enumerate(choice_sorted)}


matrix_3d = np.zeros((len(cards_sorted), len(dist_sorted), len(choice_sorted)))


for _, row in df_combined.iterrows():
    i = card_idx[row["myCard"]]-1
    j = dist_idx[row["distribution"]]
    k = choice_idx[row["choice"]]
    matrix_3d[i, j, k] += 1  


total_per_card_dist = matrix_3d.sum(axis=2, keepdims=True)

# compute percentages, avoiding division by zero
with np.errstate(divide='ignore', invalid='ignore'):
    percentage_matrix = np.divide(matrix_3d, total_per_card_dist, where=total_per_card_dist != 0)

# convert to a DataFrame for easy visualization
percentage_list = []
for i, card in enumerate(cards_sorted):
    for j, dist in enumerate(dist_sorted):
        for k, choice in enumerate(choice_sorted):
            percentage_list.append({
                "myCard": card,
                "distribution": dist,  # Now follows "uniform", "low", "high" order
                "choice": choice,
                "percentage": percentage_matrix[i, j, k]
            })

df_percentages = pd.DataFrame(percentage_list)
df_percentages

,myCard,distribution,choice,percentage
0,1,uniform,arrowdown,0.929825
1,1,uniform,arrowup,0.070175
2,1,low,arrowdown,0.871560
3,1,low,arrowup,0.128440
4,1,high,arrowdown,0.884615
5,1,high,arrowup,0.115385
6,2,uniform,arrowdown,0.943662
7,2,uniform,arrowup,0.056338
8,2,low,arrowdown,0.814815
9,2,low,arrowup,0.185185


In [6]:
np.shape(percentage_matrix)

(9, 3, 2)

In [7]:
actions = { "arrowdown": 0, "arrowup": 1}
distributions_map = { "uniform": 0, "low": 1,  "high": 2}
card_numbers = list(range(1, 10))

policy_table = percentage_matrix 

Q_table_init = np.random.normal(0, 0.01, (len(card_numbers), len(distributions_map), len(actions)))
# having a q-table based on the policies
# Q_table_init = policy_table * np.mean(Q_table_init) 
Q_table = Q_table_init.copy()

#############################################################################################
# having a q-table that starts with 0! this was not a good initilization so i changed it.
# Q_table = np.zeros((len(distributions_map), len(actions)))  # 3 distributions × 2 actions
#############################################################################################

# print("policy: \n",np.shape(policy_table))
print("\n Q_table: \n",np.shape(Q_table))




 Q_table: 
 (9, 3, 2)


In [8]:
def epsilon_greedy(Q_values, epsilon):    
    probs = np.full_like(Q_values, epsilon / Q_values.shape[-1], dtype=float)  # initialize with exploration probability
    best_actions = np.argmax(Q_values, axis=-1)  # find the best action for each state
    np.put_along_axis(probs, np.expand_dims(best_actions, axis=-1), 1 - epsilon + (epsilon / Q_values.shape[-1]), axis=-1)
    return probs



def train_rescorla_wagner(df, alpha, beta, Q_init=None):
    if Q_init is None:
        Q_init = Q_table.copy()
    Q_values = Q_init.copy()
    q_value_pairs = []
    choices = []
    predicted_probs = []
    distributions = []
    card_numbers = []
    
    for _, row in df.iterrows():
        action = actions[row["choice"]] 
        distribution = distributions_map[row["distribution"]] 
        card_number = row["myCard"]-1 # since I'm using this as an index! I need to do -1 to make the 1 to 9 cards come to 0 to 8
        reward = 0.5 if row["outcome"] == "win" else -0.5


        probs = epsilon_greedy(Q_values, beta)
        predicted_probs.append(probs[card_number][distribution][action])
        
        prediction_error = reward - Q_values[card_number][distribution][action]
        Q_values[card_number][distribution][action] += alpha * prediction_error
        
        q_value_pairs.append(Q_values.copy())
        choices.append(action)
        distributions.append(distribution)
        card_numbers.append(card_number)
        

    return np.array(q_value_pairs), np.array(choices), np.array(predicted_probs), np.array(distributions), np.array(card_numbers)


# this is for the sake of parallel computing
def compute_log_likelihood(alpha, beta, df_all, Q_table):
    Q_init_participant = Q_table.copy()
    q_values, choices, predicted_probs, distributions, card_numbers = train_rescorla_wagner(df_all, alpha, beta, Q_init=Q_init_participant.copy())
    
    predicted_probs = np.clip(predicted_probs, 1e-6, 1)  # prevent log(0)
    log_likelihood = np.sum(np.log(predicted_probs))
    
    return (alpha, beta, log_likelihood)


In [9]:
num_of_samples = 100
# num_of_samples = 1000
alpha_min = 0.01
alpha_max = 1
beta_min = 0.01
beta_max  = 1
alpha_samples = np.random.uniform(alpha_min, alpha_max + np.finfo(float).eps, num_of_samples)
beta_samples = np.random.uniform(beta_min, beta_max + np.finfo(float).eps, num_of_samples)

In [10]:
BIC_models = []
AIC_models = []
best_alpha_models = []
best_beta_models = []
accuracy_models = []
precision_models = []
sensitivity_recall_models = []
specificity_models = []
f1_score_models = []
mcFadden_r2_models = []
r2_models = []

for idx, df_all in enumerate(dataframes):
    Q_init_participant = Q_table.copy()
    
    df_all = df_all[df_all['outcome'].str.lower() != 'na'].reset_index(drop=True)


    def objective(trial):
        alpha = trial.suggest_float("alpha", alpha_min, alpha_max)
        beta  = trial.suggest_float("beta", beta_min, beta_max)

        # negative log-likelihood (Optuna minimises)
        _, _, ll = compute_log_likelihood(alpha, beta,
                                        df_all,
                                        Q_init_participant.copy())
        return -ll

    study = optuna.create_study(direction="minimize",
                                sampler=optuna.samplers.TPESampler(seed=42))
    study.optimize(objective, n_trials=400, n_jobs=-1)




    best_alpha = study.best_params["alpha"]
    best_beta  = study.best_params["beta"]
    best_log_likelihood = -study.best_value

    # keep this for plotting later
    results_df = study.trials_dataframe()
    results_df["alpha"] = results_df["params_alpha"]
    results_df["beta"]  = results_df["params_beta"]
    results_df["log_likelihood"] = -results_df["value"]

    # model prediction 
    
    q_values, choices, predicted_probs, distributions, card_numbers = train_rescorla_wagner(df_all, best_alpha, best_beta, Q_init=Q_init_participant.copy())
    
    
    predicted_choices = []
    for trial in range(len(card_numbers)):
        test_action_probs = epsilon_greedy(q_values[trial], best_beta)
        p_arrowup = test_action_probs[card_numbers[trial]][distributions[trial]][actions["arrowup"]]
        p_arrow_down = test_action_probs[card_numbers[trial]][distributions[trial]][actions["arrowdown"]]
        # choosing 1 or 0 based on the softmax probabilities:
        predicted_choices.append(np.random.choice([1, 0], p=[p_arrowup, p_arrow_down]))

    # finding out model total reward based on the model's predicted choices
    total_reward = [] 
    for i in range(len(predicted_choices)):
        if len(total_reward)> 0:
            last_reward = total_reward[-1]  #  the last reward value
        else:
            last_reward = 10 # initial reward is $10
        
        if ((df_all.loc[i, 'myCard'] > df_all.loc[i, 'yourCard'] and predicted_choices[i] == 1) or
            (df_all.loc[i, 'myCard'] < df_all.loc[i, 'yourCard'] and predicted_choices[i] == 0)):
            total_reward.append(last_reward + 0.5)
        else:
            total_reward.append(last_reward - 0.5)

    
   
       # confusion matrix:
    conf_matrix = confusion_matrix(choices, predicted_choices)
    TN, FP, FN, TP = conf_matrix.ravel()  # unpacking the confusion matrix
    # acc
    accuracy = (TP + TN) / (TP + TN + FP + FN)
    # precision: From the ones that we’ve announced them as up/down, which ones are really up/down?
    precision = TP / (TP + FP) if (TP + FP) != 0 else 0
    # recall or sensitivity : true positive rate
    sensitivity_recall = TP / (TP + FN) if (TP + FN) != 0 else 0
    # specificity : true negative rate
    specificity = TN / (TN + FP) if (TN + FP) != 0 else 0
    # f1 Score
    f1_score = 2 * (precision * sensitivity_recall) / (precision + sensitivity_recall) if (precision + sensitivity_recall) != 0 else 0

    
    # bayes information criterion:
    n_trials = len(df_all)
    k = 2  # number of free parameters: alpha and beta
    BIC = k * np.log(n_trials) - 2 * best_log_likelihood # this is BIC formula based on the log lkelihode I found before

    

    # Akaike  information criterion(AIC):
    AIC = 2 * k - 2 * best_log_likelihood 


    # mcFadden r-squared:
    p_null = np.mean(choices)  # probability of choosing "1" in the dataset
    log_likelihood_null = np.sum(choices * np.log(p_null) + (1 - choices) * np.log(1 - p_null))
    mcFadden_r2 = 1 - (best_log_likelihood / log_likelihood_null)

    # r-squared
    r2 = r2_score(choices, predicted_choices)
    
    # saving models evaluation variables:
    best_alpha_models.append(best_alpha)
    best_beta_models.append(best_beta)
    BIC_models.append(BIC)
    AIC_models.append(AIC)
    accuracy_models.append(accuracy)
    precision_models.append(precision)
    sensitivity_recall_models.append(sensitivity_recall)
    specificity_models.append(specificity)
    f1_score_models.append(f1_score)
    mcFadden_r2_models.append(mcFadden_r2)
    r2_models.append(r2)


    ###########################################################################################
    ## visulization
    ###########################################################################################

    fig, axes = plt.subplots(1, 3, figsize=(19, 6))

    plots_smooth_level = 20


#############################################
    # Density Plot (KDE)
    sns.kdeplot(
        x=results_df["alpha"], 
        y=results_df["beta"], 
        fill=True, 
        cmap="viridis", 
        ax=axes[0], 
        bw_adjust=1.8,  # Increase for smoother density
        levels=plots_smooth_level,  # More contour levels
        thresh=0  # Ensure density is plotted across all values
    )
    mappable = axes[0].collections[0]
    cbar = fig.colorbar(mappable, ax=axes[0], label="density", fraction=0.046, pad=0.04)  
    cbar.ax.yaxis.set_major_formatter(mticker.FormatStrFormatter('%.2f'))  # 2 decimal places
    cbar.ax.set_ylabel("density", fontsize=12, fontweight='bold')
    cbar.ax.tick_params(labelsize=12)

    axes[0].set_xlim(alpha_min, alpha_max)
    axes[0].set_ylim(beta_min, beta_max)
    axes[0].set_xlabel("learning rate (α)", fontsize=14, fontweight='bold')
    axes[0].set_ylabel("epsilon (ε)", fontsize=14, fontweight='bold')
    axes[0].set_title("density of α and ε joint probability", fontsize=16, fontweight='bold')
    axes[0].tick_params(axis='both', labelsize=14)
#############################################


#############################################
    # Log Likelihood 

    
    alpha_step = 0.1
    beta_step = 0.1
    alpha_bins = np.arange(alpha_min, alpha_max+ alpha_step, alpha_step)  # bins from 0 to 1 with step 0.1
    beta_bins = np.arange(beta_min, beta_max + beta_step, beta_step)       # bins from 0 to 1 with step 0.1

    results_df["alpha_binned"] = pd.cut(results_df["alpha"], bins=alpha_bins, labels=alpha_bins[:-1], include_lowest=True)
    results_df["beta_binned"] = pd.cut(results_df["beta"], bins=beta_bins, labels=beta_bins[:-1], include_lowest=True)

    heatmap_data = results_df.groupby(
    ["beta_binned", "alpha_binned"], observed=False)["log_likelihood"].mean().unstack()

    heatmap_data.index = heatmap_data.index.astype(float)
    heatmap_data.columns = heatmap_data.columns.astype(float)

    sns.heatmap(
        heatmap_data, 
        cmap="Blues", 
        cbar=True,
        ax=axes[1]
    )
    axes[1].set_xticks(np.arange(len(heatmap_data.columns)))  
    axes[1].set_xticklabels([f"{x:.1f}" for x in heatmap_data.columns], rotation=45)

    axes[1].set_yticks(np.arange(len(heatmap_data.index))) 
    axes[1].set_yticklabels([f"{x:.1f}" for x in heatmap_data.index]) 


    axes[1].set_xlabel("learning rate (α)", fontsize=14, fontweight='bold')
    axes[1].set_ylabel("epsilon (ε)", fontsize=14, fontweight='bold')
    axes[1].set_title("log likelihood for combinations of α and ε", fontsize=16, fontweight='bold')
    axes[1].tick_params(axis='both', labelsize=14)
    axes[1].invert_yaxis()


    

#############################################


#############################################
    # Confusion Matrix
    heatmap_cmap_color = mcolors.LinearSegmentedColormap.from_list("warm_red", ["#fff5e6", "#ff5733"])
    sns.heatmap(
        conf_matrix, annot=True, fmt="d", cmap=heatmap_cmap_color,
        xticklabels=["arrowdown", "arrowup"], 
        yticklabels=["arrowdown", "arrowup"], 
        ax=axes[2], 
        cbar=False
    )

    axes[2].set_xlabel("prediction", fontsize=14, fontweight='bold')
    axes[2].set_ylabel("true label", fontsize=14, fontweight='bold')
    axes[2].set_title(f"confusion matrix (α={best_alpha:.2f}, ε={best_beta:.2f})", fontsize=16, fontweight='bold')
    axes[2].tick_params(axis='both', labelsize=14)


#############################################
    # saving figures
    plt.tight_layout(rect=[0, 0, 1, 0.9]) 
    fig.suptitle(f'participant {idx}', fontsize=18, fontweight='bold', y=0.95)

    filename = os.path.join(output_dir_plots, f"plot_{participants[idx]}.pdf")
    plt.savefig(filename, format='pdf')
    plt.close(fig)

    print(f"saved: {filename}")


#############################################

    # saving model behavior
    q_values_reshaped = [q_values[i].tolist() for i in range(n_trials)]  # convert each (9,3,2) array into a list format

    df_model_behavior = pd.DataFrame({
        "model_choices": predicted_choices,
        "participant_choices": choices,
        "model_total_reward": total_reward,
        "participant_total_reward": df_all["totalReward"],
        "q_val": q_values_reshaped  
    })

    file_path = os.path.join(output_dir_model_behavior, f"model_behavior_{participants[idx]}.csv")
    df_model_behavior.to_csv(file_path, index=False)



[I 2025-06-18 15:08:33,018] A new study created in memory with name: no-name-e76c8c33-a77f-4539-beec-ecd04b21a9e1
[I 2025-06-18 15:08:34,061] Trial 6 finished with value: 256.90249772178277 and parameters: {'alpha': 0.7979664777095623, 'beta': 0.05519216987467371}. Best is trial 6 with value: 256.90249772178277.
[I 2025-06-18 15:08:34,092] Trial 3 finished with value: 249.403388356969 and parameters: {'alpha': 0.42239474526479576, 'beta': 0.0555255823012021}. Best is trial 3 with value: 249.403388356969.
[I 2025-06-18 15:08:34,116] Trial 12 finished with value: 161.23346387599057 and parameters: {'alpha': 0.7457449228041153, 'beta': 0.3408156393593384}. Best is trial 12 with value: 161.23346387599057.
[I 2025-06-18 15:08:34,707] Trial 0 finished with value: 172.48654309752584 and parameters: {'alpha': 0.935216724373399, 'beta': 0.8703312136763104}. Best is trial 12 with value: 161.23346387599057.
[I 2025-06-18 15:08:34,853] Trial 2 finished with value: 177.3664657614477 and parameters:

saved: 13_RL_agent_TDlearn_output\plots\plot_02_01_2025_13_21_03.pdf


[I 2025-06-18 15:09:07,401] Trial 12 finished with value: 171.4525872872262 and parameters: {'alpha': 0.3663623280090508, 'beta': 0.8287689292178961}. Best is trial 12 with value: 171.4525872872262.
[I 2025-06-18 15:09:07,425] Trial 1 finished with value: 223.77434681504306 and parameters: {'alpha': 0.7920222982219405, 'beta': 0.1347953780643194}. Best is trial 12 with value: 171.4525872872262.
[I 2025-06-18 15:09:07,444] Trial 2 finished with value: 205.37219186657293 and parameters: {'alpha': 0.45904834714013965, 'beta': 0.1817044204830333}. Best is trial 12 with value: 171.4525872872262.
[I 2025-06-18 15:09:07,455] Trial 17 finished with value: 180.84444452882175 and parameters: {'alpha': 0.856936920730205, 'beta': 0.2890631101424069}. Best is trial 12 with value: 171.4525872872262.
[I 2025-06-18 15:09:07,463] Trial 15 finished with value: 186.13953659360806 and parameters: {'alpha': 0.5700329152001128, 'beta': 0.991043824051548}. Best is trial 12 with value: 171.4525872872262.
[I 2

saved: 13_RL_agent_TDlearn_output\plots\plot_03_02_2025_15_29_50.pdf


[I 2025-06-18 15:09:40,597] Trial 17 finished with value: 137.85200705762958 and parameters: {'alpha': 0.20750099449883422, 'beta': 0.42386547817943226}. Best is trial 17 with value: 137.85200705762958.
[I 2025-06-18 15:09:40,815] Trial 0 finished with value: 156.705698107402 and parameters: {'alpha': 0.6365346358193855, 'beta': 0.5609213458335947}. Best is trial 17 with value: 137.85200705762958.
[I 2025-06-18 15:09:40,890] Trial 1 finished with value: 157.23954803378328 and parameters: {'alpha': 0.6399750582500862, 'beta': 0.4729693103291655}. Best is trial 17 with value: 137.85200705762958.
[I 2025-06-18 15:09:40,894] Trial 12 finished with value: 136.63834224209023 and parameters: {'alpha': 0.17335235653935566, 'beta': 0.3805801352026496}. Best is trial 12 with value: 136.63834224209023.
[I 2025-06-18 15:09:40,915] Trial 5 finished with value: 147.50084601841553 and parameters: {'alpha': 0.42755963116555745, 'beta': 0.33164205491157067}. Best is trial 12 with value: 136.63834224209

...

saved: 13_RL_agent_TDlearn_output\plots\plot_14_03_2025_16_05_47.pdf


[I 2025-06-18 15:11:26,685] Trial 0 finished with value: 161.7498145389415 and parameters: {'alpha': 0.590465017361549, 'beta': 0.7269508740099601}. Best is trial 0 with value: 161.7498145389415.
[I 2025-06-18 15:11:26,731] Trial 9 finished with value: 171.43166002261412 and parameters: {'alpha': 0.8557562666152023, 'beta': 0.8561553489848063}. Best is trial 0 with value: 161.7498145389415.
[I 2025-06-18 15:11:26,754] Trial 4 finished with value: 155.86466840105072 and parameters: {'alpha': 0.6743388081038483, 'beta': 0.5687291002830601}. Best is trial 4 with value: 155.86466840105072.
[I 2025-06-18 15:11:26,756] Trial 13 finished with value: 155.98584537301394 and parameters: {'alpha': 0.167430715768068, 'beta': 0.39825079699478927}. Best is trial 4 with value: 155.86466840105072.
[I 2025-06-18 15:11:26,780] Trial 6 finished with value: 160.88753340341333 and parameters: {'alpha': 0.9473070766380597, 'beta': 0.3647694776918222}. Best is trial 4 with value: 155.86466840105072.
[I 2025-

saved: 13_RL_agent_TDlearn_output\plots\plot_20_05_2025_13_31_58.pdf


[I 2025-06-18 15:11:58,482] Trial 4 finished with value: 182.25290110232413 and parameters: {'alpha': 0.13114423213714457, 'beta': 0.2803712002592921}. Best is trial 0 with value: 162.54453335877452.
[I 2025-06-18 15:11:58,522] Trial 5 finished with value: 167.99584194406236 and parameters: {'alpha': 0.45018527966025684, 'beta': 0.7886170231951684}. Best is trial 0 with value: 162.54453335877452.
[I 2025-06-18 15:11:58,582] Trial 3 finished with value: 160.6481057076881 and parameters: {'alpha': 0.712245627508662, 'beta': 0.5956573497528348}. Best is trial 3 with value: 160.6481057076881.
[I 2025-06-18 15:11:58,805] Trial 7 finished with value: 176.06382458335963 and parameters: {'alpha': 0.4081780681655435, 'beta': 0.890942707636116}. Best is trial 3 with value: 160.6481057076881.
[I 2025-06-18 15:11:59,073] Trial 2 finished with value: 180.64236572962085 and parameters: {'alpha': 0.7373127926120978, 'beta': 0.9409215683055683}. Best is trial 3 with value: 160.6481057076881.
[I 2025-0

saved: 13_RL_agent_TDlearn_output\plots\plot_28_02_2025_12_02_47.pdf


# now saving the model evaluation values

In [11]:
df_models_evaluation = pd.DataFrame({
    "participants": participants,
    "best_alpha": best_alpha_models,
    "best_beta": best_beta_models,
    "BIC": BIC_models,
    "AIC": AIC_models,
    "accuracy": accuracy_models,
    "precision": precision_models,
    "sensitivity_recall": sensitivity_recall_models,
    "specificity": specificity_models,
    "f1_score": f1_score_models,
    "mcFadden_r2": mcFadden_r2_models,
    "r2": r2_models
})

file_path = os.path.join(output_dir_model_evaluation, "models_evaluation.csv")
df_models_evaluation.to_csv(file_path, index=False)